In [6]:
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task

In [1]:
import cv2 as cv
from web3 import Web3
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import json
import mediapipe as mp
from tensorflow import keras
from tensorflow.keras import layers
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_LEFT_EYE
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_RIGHT_EYE
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_FACE_OVAL
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_NOSE
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_LIPS
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_RIGHT_IRIS
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_LEFT_IRIS
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_LEFT_EYEBROW
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_RIGHT_EYEBROW

In [2]:
def calculate_area(cluster_coords):
    # Calculate the area of the cluster using the Shoelace formula.
    # Assumes cluster_coords is a list of (x, y, z) coordinate tuples.
    n = len(cluster_coords)
    area = 0
    for i in range(n):
        j = (i + 1) % n
        area += cluster_coords[i][0] * cluster_coords[j][1]
        area -= cluster_coords[j][0] * cluster_coords[i][1]
    area = abs(area) / 2
    return area

In [3]:
def calculate_perimeter(cluster_coords):
    # Calculate the perimeter of the cluster as the sum of distances between consecutive points.
    # Assumes cluster_coords is a list of (x, y, z) coordinate tuples.
    n = len(cluster_coords)
    perimeter = 0
    for i in range(n):
        j = (i + 1) % n
        dist = np.linalg.norm(np.array(cluster_coords[i]) - np.array(cluster_coords[j]))
        perimeter += dist
    return perimeter

In [4]:
def calculate_aspect_ratio(cluster_coords):
    # Calculate the aspect ratio of the cluster as the ratio of width to height.
    # Assumes cluster_coords is a list of (x, y, z) coordinate tuples.
    xs = [coord[0] for coord in cluster_coords]
    ys = [coord[1] for coord in cluster_coords]
    width = max(xs) - min(xs)
    height = max(ys) - min(ys)
    aspect_ratio = width / height
    return aspect_ratio

In [5]:
def calculate_geometrical_features(cluster_coords):
    # Calculate geometrical features based on the cluster coordinates
    centroid = np.mean(cluster_coords, axis=0)
    area = calculate_area(cluster_coords)
    perimeter = calculate_perimeter(cluster_coords)
    aspect_ratio = calculate_aspect_ratio(cluster_coords)

    geometrical_features = {
        'centroid': centroid,
        'area': area,
        'perimeter': perimeter,
        'aspect_ratio': aspect_ratio,
    }

    return geometrical_features

In [6]:
def extract_cluster_features(result):
    # Extract the coordinates of specific facial landmarks for feature extraction.
    left_eye_coords = []
    right_eye_coords = []
    nose_coords = []
    lips_coords = []
    left_eyebrow_coords = []
    right_eyebrow_coords = []
    face_oval_coords = []
    left_iris_coords = []
    right_iris_coords = []

    for face_landmarks in result.face_landmarks:
        left_eye_landmarks = [face_landmarks[start:end+1] for start, end in FACEMESH_LEFT_EYE]
        # Access the values of the left eye landmarks
        for landmarks in left_eye_landmarks:
            for landmark in landmarks:
                left_eye_coords.append((landmark.x, landmark.y, landmark.z))

        right_eye_landmarks = [face_landmarks[start:end+1] for start, end in FACEMESH_RIGHT_EYE]
        # Access the values of the right eye landmarks
        for landmarks in right_eye_landmarks:
            for landmark in landmarks:
                right_eye_coords.append((landmark.x, landmark.y, landmark.z)) 

        nose_landmarks = [face_landmarks[start:end+1] for start, end in FACEMESH_NOSE]
        # Access the values of the nose landmarks
        for landmarks in nose_landmarks:
            for landmark in landmarks:
                nose_coords.append((landmark.x, landmark.y, landmark.z)) 

        lips_landmarks = [face_landmarks[start:end+1] for start, end in FACEMESH_LIPS]
        # Access the values of the lips landmarks
        for landmarks in lips_landmarks:
            for landmark in landmarks:
                lips_coords.append((landmark.x, landmark.y, landmark.z)) 

        left_eyebrow_landmarks = [face_landmarks[start:end+1] for start, end in FACEMESH_LEFT_EYEBROW]
        # Access the values of the left eyebrow landmarks
        for landmarks in left_eyebrow_landmarks:
            for landmark in landmarks:
                left_eyebrow_coords.append((landmark.x, landmark.y, landmark.z)) 

        right_eyebrow_landmarks = [face_landmarks[start:end+1] for start, end in FACEMESH_RIGHT_EYEBROW]
        # Access the values of the right eyebrow landmarks
        for landmarks in right_eyebrow_landmarks:
            for landmark in landmarks:
                right_eyebrow_coords.append((landmark.x, landmark.y, landmark.z)) 

        face_oval_landmarks = [face_landmarks[start:end+1] for start, end in FACEMESH_FACE_OVAL]
        # Access the values of the face oval landmarks
        for landmarks in face_oval_landmarks:
            for landmark in landmarks:
                face_oval_coords.append((landmark.x, landmark.y, landmark.z)) 

        left_iris_landmarks = [face_landmarks[start:end+1] for start, end in FACEMESH_LEFT_IRIS]
        # Access the values of the left iris landmarks
        for landmarks in left_iris_landmarks:
            for landmark in landmarks:
                left_iris_coords.append((landmark.x, landmark.y, landmark.z)) 

        right_iris_landmarks = [face_landmarks[start:end+1] for start, end in FACEMESH_RIGHT_IRIS]
        # Access the values of the right iris landmarks
        for landmarks in right_iris_landmarks:
            for landmark in landmarks:
                right_iris_coords.append((landmark.x, landmark.y, landmark.z))          

    # Calculate geometrical features for each cluster
    left_eye_geometrical_features = calculate_geometrical_features(left_eye_coords)
    right_eye_geometrical_features = calculate_geometrical_features(right_eye_coords)
    nose_geometrical_features = calculate_geometrical_features(nose_coords)
    lips_geometrical_features = calculate_geometrical_features(lips_coords)
    left_eyebrow_geometrical_features = calculate_geometrical_features(left_eyebrow_coords)
    right_eyebrow_geometrical_features = calculate_geometrical_features(right_eyebrow_coords)
    face_oval_geometrical_features = calculate_geometrical_features(face_oval_coords)
    left_iris_geometrical_features = calculate_geometrical_features(left_iris_coords)
    right_iris_geometrical_features = calculate_geometrical_features(right_iris_coords)

    # Return the extracted features for the individual clusters
    cluster_features = {
        'left_eye': left_eye_geometrical_features,
        'right_eye': right_eye_geometrical_features,
        'nose': nose_geometrical_features,
        'lips': lips_geometrical_features,
        'left_eyebrow': left_eyebrow_geometrical_features,
        'right_eyebrow': right_eyebrow_geometrical_features,
        'face_oval': face_oval_geometrical_features,
        'left_iris': left_iris_geometrical_features,
        'right_iris': right_iris_geometrical_features
    }

    return cluster_features

In [7]:
def draw_landmarks_on_image(image, landmarks):
  # Draw landmarks on the image
  for landmark in landmarks:
    x = int(landmark.x * image.shape[1])
    y = int(landmark.y * image.shape[0])
    cv.circle(image, (x, y), 1, (40, 25, 52), -1)
  return image

In [8]:
# Define the Generator and Discriminator models
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Linear(input_dim, 1)

    def forward(self, x):
        return torch.sigmoid(self.fc(x))

class GANModel:
    def __init__(self, input_dim, output_dim):
        self.generator = Generator(input_dim, output_dim)
        self.discriminator = Discriminator(output_dim)
        self.criterion = nn.MSELoss()
        self.generator_optimizer = optim.Adam(self.generator.parameters(), lr=0.0002)
        self.discriminator_optimizer = optim.Adam(self.discriminator.parameters(), lr=0.0002)
        self.scheduler_G = torch.optim.lr_scheduler.ExponentialLR(self.generator_optimizer, gamma=0.98)
        self.scheduler_D = torch.optim.lr_scheduler.ExponentialLR(self.discriminator_optimizer, gamma=0.98)

    def generate_noise(self, shape):
        return torch.randn(shape)

    def add_noise(self, generatedTensor, realTensor):
        return generatedTensor+realTensor

    def remove_noise(self, generatedTensor, noisyTensor):
        return noisyTensor-generatedTensor

In [9]:
def tensorToDictionary(tensorObject):
    tensorList = np.array(tensorObject)
    flattenedTensorList = tensorList.flatten()

    tensorDictionary = {
        'left_eye': {
            'centroid': flattenedTensorList[:3].tolist(),
            'area': flattenedTensorList[3],
            'perimeter': flattenedTensorList[4],
            'aspect_ratio': flattenedTensorList[5]
        },
        'right_eye': {
            'centroid': flattenedTensorList[6:9].tolist(),
            'area': flattenedTensorList[9],
            'perimeter': flattenedTensorList[10],
            'aspect_ratio': flattenedTensorList[11]
        },
        'nose': {
            'centroid': flattenedTensorList[12:15].tolist(),
            'area': flattenedTensorList[15],
            'perimeter': flattenedTensorList[16],
            'aspect_ratio': flattenedTensorList[17]
        },
        'lips': {
            'centroid': flattenedTensorList[18:21].tolist(),
            'area': flattenedTensorList[21],
            'perimeter': flattenedTensorList[22],
            'aspect_ratio': flattenedTensorList[23]
        },
        'left_eyebrow': {
            'centroid': flattenedTensorList[24:27].tolist(),
            'area': flattenedTensorList[27],
            'perimeter': flattenedTensorList[28],
            'aspect_ratio': flattenedTensorList[29]
        },
        'right_eyebrow': {
            'centroid': flattenedTensorList[30:33].tolist(),
            'area': flattenedTensorList[33],
            'perimeter': flattenedTensorList[34],
            'aspect_ratio': flattenedTensorList[35]
        },
        'face_oval': {
            'centroid': flattenedTensorList[36:39].tolist(),
            'area': flattenedTensorList[39],
            'perimeter': flattenedTensorList[40],
            'aspect_ratio': flattenedTensorList[41]
        },
        'left_iris': {
            'centroid': flattenedTensorList[42:45].tolist(),
            'area': flattenedTensorList[45],
            'perimeter': flattenedTensorList[46],
            'aspect_ratio': flattenedTensorList[47]
        },
        'right_iris': {
            'centroid': flattenedTensorList[48:51].tolist(),
            'area': flattenedTensorList[51],
            'perimeter': flattenedTensorList[52],
            'aspect_ratio': flattenedTensorList[53]
        }
    }

    return tensorDictionary

In [11]:
def face_landmark_extraction(image_path):
    # Load the image
    readImg = cv.imread(image_path)
    image = mp.Image.create_from_file(image_path)

    # STEP 2: Create a FaceLandmarker object
    base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
    options = vision.FaceLandmarkerOptions(base_options=base_options,
                                           output_face_blendshapes=True,
                                           output_facial_transformation_matrixes=True,
                                           num_faces=1)
    detector = vision.FaceLandmarker.create_from_options(options)

    # STEP 3: Detect face landmarks from the input image.
    detection_result = detector.detect(image)
    
    # Check if a face was detected
    if detection_result.face_landmarks:
        cluster_features = extract_cluster_features(detection_result)

        # Extract the coordinates of the landmarks
        landmarks = [landmark for face_landmarks in detection_result.face_landmarks for landmark in face_landmarks]
        
        # Draw landmarks on the image
        annotated_image = draw_landmarks_on_image(readImg.copy(), landmarks)

    else:
        print("No face detected in the image.")
    return cluster_features

In [12]:
def print_dictionary(dictionary):
    for key, value in dictionary.items():
        print(key)
        print('-' * len(key))
        print(value)
        print()

In [13]:
# Determine the input and output dimensions for the GAN model
input_dim = 54
output_dim = 54

# Create the GAN model
gan_model = GANModel(input_dim, output_dim)

generator = Generator(input_dim, output_dim)
discriminator = Discriminator(output_dim)
generator.load_state_dict(torch.load(r'G:\\SFF\\SFF\\generator_model.pth'))
discriminator.load_state_dict(torch.load(r'G:\\SFF\\SFF\\discriminator_model.pth'))
gan_model.generator = generator
gan_model.discriminator = discriminator

In [14]:
noise = gan_model.generate_noise((1, input_dim))
generated_data = gan_model.generator(noise)
generated_data = generated_data.detach()

In [15]:
image_path = r"G:\\SFF\\Diagrams\\download (3).png"
faceFeatures = face_landmark_extraction(image_path)

In [16]:
faceFeatures_str = str(faceFeatures)
# Remove 'array()' from the string
faceFeatures_str = faceFeatures_str.replace('array(', '').replace(')', '')

# Replace single quotes with double quotes
faceFeatures_str = faceFeatures_str.replace("'", '"')

faceFeatures_dict = json.loads(faceFeatures_str)

# Convert the dictionary into a tensor
faceFeatures_list = []
for key, value in faceFeatures_dict.items():
    faceFeatures_list.extend(value['centroid'])
    faceFeatures_list.append(value['area'])
    faceFeatures_list.append(value['perimeter'])
    faceFeatures_list.append(value['aspect_ratio'])
faceFeatures_tensor = torch.tensor(faceFeatures_list)

In [17]:
noisyFaceFeatures_tensor = gan_model.add_noise(generated_data, faceFeatures_tensor)
noisyFaceFeatures_tensor = noisyFaceFeatures_tensor[0]

In [19]:
# Define the neural network architecture for encryption (encryptor)
encryptor = keras.Sequential([
    layers.Input(shape=(54,)),  # Input shape matches the number of features in your data
    layers.Dense(32, activation='relu'),  # Hidden layer with ReLU activation
    layers.Dense(16, activation='relu'),  # Hidden layer with ReLU activation
    layers.Dense(8, activation='relu'),   # Hidden layer with ReLU activation
    layers.Dense(4, activation='linear')  # Output layer with linear activation
])

# Define the neural network architecture for decryption (decryptor)
decryptor = keras.Sequential([
    layers.Input(shape=(4,)),               # Input shape matches the size of the encoded data (8 features)
    layers.Dense(8, activation='relu'),
    layers.Dense(16, activation='relu'),   # Hidden layer with ReLU activation
    layers.Dense(32, activation='relu'),   # Hidden layer with ReLU activation
    layers.Dense(54, activation='linear')  # Output layer with linear activation
])

# Combine the encryptor and decryptor into an autoencoder
autoencoder = keras.Sequential([encryptor, decryptor])

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [20]:
# Call the model once to create its variables
dummy_input = np.zeros((1, 54))  # Create a dummy input with the same shape as your data
autoencoder.predict(dummy_input)

# Load the weights of the autoencoder model
autoencoder.load_weights(r'G:\\SFF\\SFF\\autoencoder_weights.h5')

1/1 [==============================] - 0s 272ms/step


In [21]:
noisyFaceFeatures_list = noisyFaceFeatures_tensor.tolist()
noisyFaceFeatures_numpy = np.array(noisyFaceFeatures_list)
noisyFaceFeatures_numpy = noisyFaceFeatures_numpy.reshape(1, 54)

# Encrypt data using the encryptor
encrypted_noisyFaceFeatures = encryptor.predict(noisyFaceFeatures_numpy)

1/1 [==============================] - 0s 76ms/step


In [22]:
encrypted_noisyFaceFeatures = encrypted_noisyFaceFeatures.reshape(4)
encrypted_noisyFaceFeatures_list = (encrypted_noisyFaceFeatures*1000000000000000).tolist()
faceFeatures_intList = []
for i in encrypted_noisyFaceFeatures_list:
    faceFeatures_intList.append(int(i))

In [23]:
# Connect to the Ethereum network
web3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
web3.isConnected()

# Load the smart contract ABI
contract_abi = [
	{
		"inputs": [
			{
				"internalType": "int256[]",
				"name": "elements",
				"type": "int256[]"
			}
		],
		"name": "addElements",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "int256[]",
				"name": "input",
				"type": "int256[]"
			}
		],
		"name": "compareElements",
		"outputs": [
			{
				"components": [
					{
						"internalType": "int256",
						"name": "bestMatchIndex",
						"type": "int256"
					},
					{
						"internalType": "bytes32",
						"name": "bestMatchId",
						"type": "bytes32"
					}
				],
				"internalType": "struct SecureArray.MatchResult",
				"name": "",
				"type": "tuple"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getArrayByIndex",
		"outputs": [
			{
				"internalType": "int256[]",
				"name": "",
				"type": "int256[]"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "getArraysCount",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "getLatestArray",
		"outputs": [
			{
				"internalType": "int256[]",
				"name": "",
				"type": "int256[]"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]

# Contract address of the deployed smart contract
contract_address = "0x896be1E5816216d9CBA8d78fb0994625105d2725"

# Your Ethereum wallet private key
private_key = "0x84488dc070916f80ee26c361ab5f2790fbab97d53516fc686c73d2849284c6e9"

# Create a contract instance
contract = web3.eth.contract(address=contract_address, abi=contract_abi)

# Build the transaction
transaction = contract.functions.addElements(faceFeatures_intList).buildTransaction({
    'gasPrice': web3.eth.gas_price,
    'nonce': web3.eth.get_transaction_count('0xB1d8EdbD86cf4EF4cc8A04C560cF84a3a08886af'),
    'from': '0xB1d8EdbD86cf4EF4cc8A04C560cF84a3a08886af',
    'chainId': 1337  # Replace with the appropriate chain ID
})

# Sign the transaction
signed_txn = web3.eth.account.sign_transaction(transaction, private_key=private_key)

# Send the signed transaction
transaction_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)

# Wait for the transaction to be mined
transaction_receipt = web3.eth.wait_for_transaction_receipt(transaction_hash)

# Check if the transaction was successful
if transaction_receipt.status == 1:
    print("Transaction successful.")
else:
    print("Transaction failed.")

ReadTimeout: HTTPConnectionPool(host='127.0.0.1', port=8545): Read timed out. (read timeout=10)

In [ ]:
# Example: Getting the count of arrays
arrays_count = contract.functions.getArraysCount().call()
print(f"Arrays Count: {arrays_count}")

KeyboardInterrupt: 

In [179]:
#Comparing elements
best_match_index, best_match_id = contract.functions.compareElements(faceFeatures_intList).call()
print(f"Best Match Index: {best_match_index}")
print(f"Best Match ID: {best_match_id}")

Best Match Index: 1
Best Match ID: b'\xb1\x0e-Rv\x12\x07;&\xee\xcd\xfdq~j2\x0c\xf4KJ\xfa\xc2\xb0s-\x9f\xcb\xe2\xb7\xfa\x0c\xf6'


In [185]:
#retrievedArray = contract.functions.getLatestArray().call()
retrievedArray = contract.functions.getArrayByIndex(1).call()

In [186]:
originalNoisyArray = []
for i in retrievedArray:
    tmp = i/1000000000000000
    originalNoisyArray.append(tmp)

In [213]:
originalNoisyNumpy = np.array(originalNoisyArray).reshape(1, 4)
decrypted_noisyFaceFeatures = decryptor.predict(originalNoisyNumpy)
decrypted_noisyFaceFeatures = decrypted_noisyFaceFeatures.reshape(54)

1/1 [==============================] - 0s 21ms/step


In [214]:
originalNoisyTensor = torch.Tensor(decrypted_noisyFaceFeatures)
noiseRemovedTensor = gan_model.remove_noise(generated_data, originalNoisyTensor)

In [215]:
retrievedDict = tensorToDictionary(noiseRemovedTensor)
print_dictionary(retrievedDict)

left_eye
--------
{'centroid': [0.02773040533065796, 0.06525057554244995, -0.04908569157123566], 'area': 0.066323794, 'perimeter': 74.69399, 'aspect_ratio': 0.21528727}

right_eye
---------
{'centroid': [-0.026955515146255493, 0.075938880443573, 0.04142708703875542], 'area': 0.004501641, 'perimeter': 85.60802, 'aspect_ratio': 0.1912381}

nose
----
{'centroid': [-0.054810404777526855, 0.07253527641296387, 0.011869680136442184], 'area': 0.10081707, 'perimeter': 223.6695, 'aspect_ratio': 0.02274704}

lips
----
{'centroid': [-0.04639354348182678, 0.07152664661407471, 0.022967277094721794], 'area': 0.7832631, 'perimeter': 504.80713, 'aspect_ratio': 0.02135384}

left_eyebrow
------------
{'centroid': [0.025379955768585205, 0.02712005376815796, -0.037541601806879044], 'area': 0.05823242, 'perimeter': 12.286117, 'aspect_ratio': 0.23462147}

right_eyebrow
-------------
{'centroid': [-0.06871122121810913, 0.06686156988143921, 0.03060988336801529], 'area': 0.016783189, 'perimeter': 14.002497, 'as